# 1. 따릉이 데이터 분석
- 대여소별 이용건수, 이동거리, 이용시간 상위 10 대여소 위치 표시
- polars 이용

#### 1) 데이터 읽어오기
- 월별 따릉이 이용 데이터
- 대여소 위치 데이터

In [75]:
import pandas as pd
import polars as pl
import folium
DATA_FILE = "./resource/seoul_bicycle_monthly.csv"
GIS_FILE = "./resource/공공자전거 대여소 정보(api수집).csv"

In [76]:
## 따릉이 이용 데이터
data_df = pl.read_csv(DATA_FILE, encoding='cp949')
data_column = ['대여일자','대여소번호','대여소명','대여구분코드','성별','연령대코드','이용건수','운동량','탄소량','이동거리','이용시간']
data_df.columns = data_column
data_df.head(2)

대여일자,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,이용시간
i64,i64,str,str,str,str,i64,f64,f64,f64,i64
202407,102,"""102. 망원역 1번출구 앞""","""일일권""","""""","""20대""",67,4304.28,40.87,176252.62,1895
202407,102,"""102. 망원역 1번출구 앞""","""일일권""","""""","""30대""",64,4142.09,38.92,167774.15,1498


In [77]:
## 대여소 위치 데이터
gis_df = pl.read_csv(GIS_FILE)
gis_column = ['RENT_NO','RENT_ID_NM','STA_LAT','STA_LONG']
gis_column_name = ['대여소번호','대여소명','위도','경도']
gis_df = gis_df[gis_column]
gis_df.columns = gis_column_name
gis_df.head(2)

대여소번호,대여소명,위도,경도
i64,str,f64,f64
108,"""108. 서교동 사거리""",37.552746,126.918617
503,"""503. 더샵스타시티 C동 앞""",37.536667,127.073593


#### 2) 대여소별 이용건수, 이용시간, 이동거리 상위 10개 리스트 가져오기

In [78]:
top10_df = data_df.group_by(
    pl.col('대여소번호')
).agg(
    pl.col('이용건수').sum().alias('총_이용건수'),
    pl.col('이동거리').sum().alias('총_이동거리'),
    pl.col('이용시간').sum().name.prefix('총_')
)
top10_df.head(2)

대여소번호,총_이용건수,총_이동거리,총_이용시간
i64,i64,f64,i64
1974,7603,1.3782e7,128774
2102,38268,1.3809e8,1009331


##### 2-1) 이용건수

In [79]:
## 컬럼명 : 이용건수, 이동거리, 이용시간
col_name = '총_이용건수'
top10_result = top10_df.sort(col_name,descending=False).limit(10)
top10_result

대여소번호,총_이용건수,총_이동거리,총_이용시간
i64,i64,f64,i64
3538,2,5993.42,52
4314,8,51221.82,328
3306,11,34773.55,249
2201,15,23198.01,300
2393,18,29031.37,304
2061,27,81064.98,630
2343,32,84010.01,671
812,33,71144.41,822
803,69,411168.55,2796


##### 2-2) 대여소 위치 정보 가져오기

In [80]:
top10_gis_result = top10_result.join(gis_df,on='대여소번호')['대여소번호','대여소명','위도','경도',col_name]
# 다른표현
# top10_result.join(gis_df,on='대여소번호').select(
#     pl.col('대여소번호'),
#     pl.col('대여소명'),
#     pl.col('위도'),
#     pl.col('경도'),
#     pl.col(col_name)
# )
top10_gis_result

대여소번호,대여소명,위도,경도,총_이용건수
i64,str,f64,f64,i64
2061,"""2061. 한강 현대아파트 건너편""",37.505928,126.969231,27
2393,"""2393. 구룡사 앞 교차로 (보도육교)""",37.472,127.051338,18
3538,"""3538. 서울숲 IT캐슬""",37.550892,127.044762,2
3306,"""3306.서울신우초등학교""",37.462147,126.931778,11
4314,"""4314. 탑성마을 버스정거장 옆""",37.459167,127.055443,8
803,"""803. 한남초교 앞 보도육교""",37.538139,127.004097,69
812,"""812. 용산전쟁기념관""",37.53484,126.977661,33
4348,"""4348. 사당역 1번출구""",37.475979,126.98391,75
2201,"""2201. 본마을 정류소 앞""",37.453281,127.051483,15


##### 2-3) 맵
- 리스트의 위도와 경도의 중간값 구하기
- 맵에 위도 경도, 마크 추가

In [81]:
median = top10_gis_result[['위도','경도']].median()
print(f'중심 : {median[0,0]},{median[0,1]}')

중심 : 37.473989485,127.02442932


In [82]:
map = folium.Map(location=(median[0,0],median[0,1]),zoom_start=12)
for i in range(len(top10_gis_result)):
    title = top10_gis_result[i,'대여소명']
    value = top10_gis_result[i,col_name]
    name = f"{title} ({col_name}:{value})"
    folium.Marker([top10_gis_result[i,'위도'],top10_gis_result[i,'경도']],popup=f'<b>bicycle</b>',tooltip=name,
                  icon=folium.Icon(
                    color='red',
                    icon_color='white',
                    icon='info-sign'
    )).add_to(map)
map